In [ ]:
#| default_exp datasets.bapps

# BAPPS

> Building a `tf.data.Dataset` for BAPPS.

In [ ]:
#| hide
import os; os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
#| export
from pathlib import Path
from typing import List

import pandas as pd
import tensorflow as tf
import cv2

After setting up the path to the directory and loading the corresponding `.csv` file, we need to create a generator that will iterate over the dataframe, load and return a 3-tuple: `(Reference Image, Distorted Image, DMOS)`. When can the pass that generator into a `tf.data.Dataset.from_generator()` to build the `Dataset` object:

In [ ]:
#| export
class BAPPS():
    """Builder for the BAPPS dataset"""

    def __init__(self,
                 path, # Path to the root directory of the dataset.
                 num_parallel_calls: int = tf.data.AUTOTUNE, # Number of parallel calls when loading the images.
                 ):
        self.path_root = Path(path) if isinstance(path, str) else path
        self.path_csv = self.path_root/"data_bapps_extra.csv"
        self.data = self.load_data(self.path_csv)
        self.paths_ref = str(self.path_root) + "/" + self.data.Reference
        self.paths_dist_0 = str(self.path_root) + "/" + self.data.Image_0
        self.paths_dist_1 = str(self.path_root) + "/" + self.data.Image_1
        self.num_parallel_calls = num_parallel_calls

    @property
    def dataset(self):
        """tf.data.Dataset object built from the BAPPS dataset."""
        return tf.data.Dataset.from_tensor_slices((self.paths_ref, self.paths_dist_0, self.paths_dist_1, self.data.Label))\
                              .map(self.preprocess, num_parallel_calls=self.num_parallel_calls)

    @staticmethod
    def preprocess(path_ref,
                   path_dist_0,
                   path_dist_1,
                   label,
                   ):
        img_ref = tf.io.read_file(path_ref)
        img_dist_0 = tf.io.read_file(path_dist_0)
        img_dist_1 = tf.io.read_file(path_dist_1)

        img_ref = tf.image.decode_png(img_ref, channels=3)
        img_dist_0 = tf.image.decode_png(img_dist_0, channels=3)
        img_dist_1 = tf.image.decode_png(img_dist_1, channels=3)

        img_ref = tf.image.convert_image_dtype(img_ref, dtype=tf.float32)
        img_dist_0 = tf.image.convert_image_dtype(img_dist_0, dtype=tf.float32)
        img_dist_1 = tf.image.convert_image_dtype(img_dist_1, dtype=tf.float32)

        img_ref = tf.image.resize(img_ref, size=(252,252))
        img_dist_0 = tf.image.resize(img_dist_0, size=(252,252))
        img_dist_1 = tf.image.resize(img_dist_1, size=(252,252))

        return img_ref, img_dist_0, img_dist_1, label

    def load_data(self,
                  path,
                  ):
        data = pd.read_csv(self.path_csv)
        return data

In [ ]:
# l = TID2013(path = Path("/media/disk/databases/BBDD_video_image/Image_Quality/TID/TID2013"))
l = BAPPS(path = Path("/lustre/ific.uv.es/ml/uv075/Databases/IQA/BAPPS/"))

In [ ]:
l.data

,Reference,Image_0,Image_1,Preference,Label,Distortion_Type
0,2afc/val/frameinterp/ref/000000.png,2afc/val/frameinterp/p0/000000.png,2afc/val/frameinterp/p1/000000.png,0.0,0,frameinterp
1,2afc/val/frameinterp/ref/000001.png,2afc/val/frameinterp/p0/000001.png,2afc/val/frameinterp/p1/000001.png,0.0,0,frameinterp
2,2afc/val/frameinterp/ref/000002.png,2afc/val/frameinterp/p0/000002.png,2afc/val/frameinterp/p1/000002.png,0.8,1,frameinterp
3,2afc/val/frameinterp/ref/000003.png,2afc/val/frameinterp/p0/000003.png,2afc/val/frameinterp/p1/000003.png,0.8,1,frameinterp
4,2afc/val/frameinterp/ref/000004.png,2afc/val/frameinterp/p0/000004.png,2afc/val/frameinterp/p1/000004.png,0.8,1,frameinterp
...,...,...,...,...,...,...
36339,2afc/val/cnn/ref/004715.png,2afc/val/cnn/p0/004715.png,2afc/val/cnn/p1/004715.png,0.0,0,cnn
36340,2afc/val/cnn/ref/004716.png,2afc/val/cnn/p0/004716.png,2afc/val/cnn/p1/004716.png,0.6,1,cnn
36341,2afc/val/cnn/ref/004717.png,2afc/val/cnn/p0/004717.png,2afc/val/cnn/p1/004717.png,1.0,1,cnn
36342,2afc/val/cnn/ref/004718.png,2afc/val/cnn/p0/004718.png,2afc/val/cnn/p1/004718.png,0.6,1,cnn


In [ ]:
for a, b, c, d in l.dataset:
    break
assert a.shape == b.shape

2024-01-30 15:27:44.145352: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [36344]
	 [[{{node Placeholder/_3}}]]


And we can benchmark it to finish:

In [ ]:
#| eval: false
from tqdm.auto import tqdm
shapes = []
for a, b, c, d in tqdm(l.dataset): 
    shapes.append(a.shape)
    # pass

2024-01-30 15:27:44.307625: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [36344]
	 [[{{node Placeholder/_3}}]]


  0%|          | 0/36344 [00:00<?, ?it/s]